Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down to the end of the code to find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 5, a_d_init = 2,
    b_d_init = 25, a_s_init = 2, b_s_init = -3, Dflag_init = True, Sflag_init = False):
        
    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Qval_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description='Pick a Quantity:',
                                 value = Qval_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=1000,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =False)
    a_s_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_s$:',
                                 value = a_s_init,
                                 continuous_update =False)
    b_s_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description= r'$b_s$:',
                                 value = b_s_init,
                                 continuous_update =False)
    Dflag_check = widgets.Checkbox(value = Dflag_init,
                                   description='Include Demand',
                                   disabled=False,
                                   indent=True) 
    Sflag_check = widgets.Checkbox(value = Sflag_init,
                                   description='Include Supply',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Qval_slider,'max'))
    widgets.jslink((b_d_slider,'value'),(b_s_slider,'max'))

    def elasticity_plot(Qmax, Pmax, Qval, a_d, b_d, a_s, b_s, Dflag, Sflag):

        # create a quantity vector, calculate supply/demand vectors
        Q = np.arange(0,Qmax)
        P_s = a_s*Q + b_s
        P_d = -a_d*Q + b_d

        # Calculate equilibrium quantity/price
        Qeq = (b_d-b_s)/(a_s + a_d)
        Peq = a_s*Qeq + b_s

        # Calculate prices for selected value
        Pval_s = a_s*Qval + b_s
        Pval_d = -a_d*Qval + b_d

        # Create figure, plot supply/demand
        fig, ax = plt.subplots(figsize=(20,10))
        if Sflag is True:
            ax.plot(Q, P_s,'r', linewidth=2, alpha=0.6,
                    label=r'Inverse Supply $\quad P_s = a_s Q + b_s$')
        if Dflag is True:
            ax.plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                    label=r'Inverse Demand $\quad P_d = -a_d Q + b_d$')

        # Add markers for the price/quantity points, with dotted lines
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        if Sflag is True and Dflag is True:
            ax.scatter(Qeq, Peq, s=mrkrSize, c='k', 
                       label=r'Equilibrium ($Q^*$={:.2f},$P^*$={:.2f})'.format(Qeq,Peq))
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.4, 
                       label='Selection')
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.4)

            ax.plot([Qeq,Qeq],[0,Peq],'k--',linewidth=1)
            ax.plot([0,Qeq],[Peq,Peq],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,max(Pval_s,Pval_d)],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.annotate(r'$Q^*$',[Qeq,0], xytext = [Qeq+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P^*$',[0,Peq], xytext = [0.15,Peq+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is True and Dflag is False:
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_s],'k--',linewidth=1)    
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is False and Dflag is True:
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_d],'k--',linewidth=1)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        # Add elasticity annotations
        if Sflag is True:
            EsStr = r'$E_p^S = {:.2f}$'.format(Pval_s/(Qval*a_s))
            ax.annotate(EsStr,[Qval,Pval_s],
                        xytext = [Qval+0.25,Pval_s-1], 
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)
        if Dflag is True:
            EdStr = r'$E_p^D = {:.2f}$'.format(-Pval_d/(Qval*a_d))
            ax.annotate(EdStr,[Qval,Pval_d],
                        xytext = [Qval+0.25,Pval_d],
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)

        # Add legend and format axes to look nice
        if Sflag is True or Dflag is True:
            ax.legend(loc='upper center', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = Pmax, bottom = 0)
        ax.set_xlim(right = Qmax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(elasticity_plot, {'Qmax': Qmax_slider,
                                                       'Pmax': Pmax_slider,
                                                       'Qval': Qval_slider, 
                                                       'a_d': a_d_slider,
                                                       'b_d': b_d_slider,
                                                       'a_s': a_s_slider, 
                                                       'b_s': b_s_slider, 
                                                       'Dflag': Dflag_check,
                                                       'Sflag': Sflag_check})

    output = widgets.VBox([out,
                  widgets.HBox([Qval_slider,
                               Qmax_slider,
                               Pmax_slider]),
                  widgets.HBox([Dflag_check, 
                                a_d_slider, 
                                b_d_slider]),
                  widgets.HBox([Sflag_check, 
                                a_s_slider, 
                                b_s_slider])])
    display(output)

    
def tax_widget(Qmax_init = 15, Pmax_init = 30, Tval_init = 0, a_d_init = 2,
    b_d_init = 25, a_s_init = 2, b_s_init = -3, Rflag_init = False, Lflag_init = False):
        
    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Tval_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description='Set tax level:',
                                 value = Tval_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=1000,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =False)
    a_s_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_s$:',
                                 value = a_s_init,
                                 continuous_update =False)
    b_s_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description= r'$b_s$:',
                                 value = b_s_init,
                                 continuous_update =False)
    Rflag_check = widgets.Checkbox(value = Rflag_init,
                                   description='Show revenue',
                                   disabled=False,
                                   indent=True) 
    Lflag_check = widgets.Checkbox(value = Lflag_init,
                                   description='Show loss',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Tval_slider,'max'))
    widgets.jslink((b_d_slider,'value'),(b_s_slider,'max'))

    def tax_plot(Qmax, Pmax, Tval, a_d, b_d, a_s, b_s, Rflag, Lflag):

        # create a quantity vector, calculate supply/demand vectors
        Q = np.arange(0,Qmax)
        P_s = a_s*Q + b_s
        P_sT = a_s*Q + b_s + Tval
        P_d = -a_d*Q + b_d

        # Calculate equilibrium quantity/price without tax
        Qeq = (b_d-b_s)/(a_s + a_d)
        Peq = a_s*Qeq + b_s

        # Calculate quantity/prices for selected tax
        QT = (b_d-b_s-Tval)/(a_s + a_d) 
        Pval_s = a_s*QT + b_s
        Pval_d = -a_d*QT + b_d

        # Create figure, plot supply/demand curves
        fig, ax = plt.subplots(figsize=(20,10))
        ax.plot(Q, P_s,'r', linewidth=2, alpha=0.6,
                label=r'Inverse Supply, no tax $\quad P_s = a_s Q + b_s$')
        ax.plot(Q, P_sT,'r--', linewidth=2, alpha=0.6,
                label=r'Inverse Supply, with tax $\quad P_s = a_s Q + b_s + T$')
        ax.plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                label=r'Inverse Demand $\quad P_d = -a_d Q + b_d$')

        # Add markers for the price/quantity points, with dotted lines
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        ax.scatter(Qeq, Peq, s=mrkrSize, c='k', 
                   label=r'Equilibrium, no tax ($Q^*$={:.2f},$P^*$={:.2f})'.format(Qeq,Peq))
        ax.scatter(QT, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                   label='Equilibrium, with tax')
        ax.scatter(QT, Pval_d, s=mrkrSize, c='k', alpha=0.6)
        
        # Plot areas if requested
        if Rflag is True:
            ax.fill([0,QT,QT,0],[Peq,Peq,Pval_d,Pval_d],'b',alpha = 0.2,
                    label = 'Tax revenue from consumers')
            ax.fill([0,QT,QT,0],[Peq,Peq,Pval_s,Pval_s],'r',alpha = 0.2,
                    label = 'Tax revenue from suppliers')
        if Lflag is True:
            ax.fill([QT,Qeq,QT],[Pval_d,Peq,Pval_s],'k',alpha = 0.2,
                    label = 'Deadweight loss')
        
        # Add dotted lines
        ax.plot([Qeq,Qeq],[0,Peq],'k--',linewidth=1)
        ax.plot([0,Qeq],[Peq,Peq],'k--',linewidth=1)
        ax.plot([0,QT],[Pval_d,Pval_d],'k--',linewidth=1)
        ax.plot([QT,QT],[0,max(Pval_s,Pval_d)],'k--',linewidth=1)
        ax.plot([0,QT],[Pval_s,Pval_s],'k--',linewidth=1)
        ax.annotate(r'$Q^*, no tax$',[Qeq,0], xytext = [Qeq+0.15,0.25], 
                    xycoords ='data', fontsize = 25, clip_on = True)
        ax.annotate(r'$Q^*, tax$',[QT,0], xytext = [QT+0.15,0.25], 
                    xycoords ='data', fontsize = 25, clip_on = True)
        ax.annotate(r'$P^*$',[0,Peq], xytext = [0.15,Peq+0.25], 
                    xycoords ='data', fontsize = 25, clip_on = True)
        ax.annotate(r'$T={:.2f}$'.format(Tval),[0,(Pval_s+Pval_d)/2], xytext = [1,(Pval_s+Pval_d)/2], 
                    xycoords ='data', fontsize = 25, clip_on = True)
        ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                    xycoords ='data', fontsize = 25, clip_on = True)
        ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                    xycoords ='data', fontsize = 25, clip_on = True)
        
        # Add legend and format axes to look nice
        ax.legend(loc='upper right', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = Pmax, bottom = 0)
        ax.set_xlim(right = Qmax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(tax_plot, {'Qmax': Qmax_slider,
                                               'Pmax': Pmax_slider,
                                               'Tval': Tval_slider, 
                                               'a_d': a_d_slider,
                                               'b_d': b_d_slider,
                                               'a_s': a_s_slider, 
                                               'b_s': b_s_slider,
                                               'Rflag': Rflag_check,
                                               'Lflag': Lflag_check})

    output = widgets.VBox([out,
                  widgets.HBox([Tval_slider,
                               Qmax_slider,
                               Pmax_slider]),
                  widgets.HBox([Rflag_check,
                                a_d_slider, 
                                b_d_slider]),
                  widgets.HBox([Lflag_check,
                                a_s_slider, 
                                b_s_slider])])
    display(output)

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Economic applications of linear systems

The standard example is a supply and demand diagram, where the two unknown variables that we want to get are the equilibrium quantity $Q^*$ and the equilibrium price $P^*$. In order to be able to identify the values of these two variables, we need two equations. Conveniently, we can use the demand and supply curves. We are initially going to re-use the elasticities example from last week, focusing this time on solving for price/quantity.

The demand/supply equations used in the diagram are technically known as *inverse* supply/demand, as they allow you to calculate the price that consumers are willing to pay or that suppliers are willing to accept for a given quantity. These are the inverse of 'normal' demand/supply functions tell you the quantity consumers are willing to buy or suppliers are willing to provide at a given price. However, you guys know by now that if you have a linear equation with two unknowns $x$ and $y$, you can always manipulate it to get two *functions* that give you $y$ as a function of $x$, or $x$ as a function of $y$. This is what we have here.

$$\left\{\begin{aligned}
P_d & = -a_d Q_d + b_d\\
P_s & = a_s Q_s + b_s\\
\end{aligned}\right.$$

As was the case last week, note that the minus in front of $a_d$ is there to explicitly state that the demand curve is downward-sloping. This means we are assuming $a_d>0$ and $a_s>0$. So we have two unknowns ($Q^*$ and $P^*$) and two equations (Supply and Demand) that have different slopes, so we should be in the clear: we know we can solve the system. 

The first step is to assume that the *markets clear*, in other words we assume that at equilibrium all the goods produced by suppliers are bought by consumers, so $Q^* = Q_d=Q_s$. This immediately implies that the price paid by the consumer is the price received by the producer, so $P^*=P_d=P_s$. While this might seem trivial (it is) it is also an important statement: the system of equations above doesn't contain $P^*$ or $Q^*$, instead it has two different prices and two different quantities!!

As a side note, this might seem very silly and unnecessarily complicated: why do we start by subscripting quantities and prices with $d$'s and $s$'s only to remove them just after? This is because sometimes markets *don't* clear, the quantities supplied and demanded don't match (shortage or surplus), and we then don't have an equilibrium. But more on that later...

Assuming that markets clear allows us to write:

$$\left\{\begin{aligned}
P^* & = -a_d Q^* + b_d\\
P^* & = a_s Q^* + b_s\\
\end{aligned}\right.$$

At this point we can equate the right hand sides of the equations and rearrange to find the equilibrium quantity:

$$\begin{aligned}
a_s Q^* + b_s & = -a_d Q^* + b_d\\
(a_s+a_d)Q^* & = b_d - b_s\\
Q^* & = \frac{b_d - b_s}{a_s+a_d}\\
\end{aligned}$$

If we replace the parameters with the values from the diagram, we have:

$$Q^* = \frac{25 -(-3)}{2+2} = \frac{22}{4}=7$$

Replacing this in the demand equation gives us:

$$P^* = -2\times7 + 25 = -14 + 25 = 11$$

Note, you could also replace in the supply equation, which would give the same result. In any case, the important result is that we have found our equilibrium price and quantity, and this matches the diagram.


In [3]:
Elasticities_widget(Sflag_init = True)

This is a pretty standard illustration of markets, and provides a neat little illustration of how systems of equations work in economics. Feel free to play around with the diagram parameters, and check that solving the resulting system 'by hand' gives the same result.

### Extending the model

I do want to take this one step further, however, and show how we can use this basic supply/demand setup to get genuinely useful insights into how a market works. This example will also show what happens when you have more variables than equations.

The example we're going to look at is adding an as-of-yet unspecified flat tax $T$ on suppliers. This is added to the price per unit that the suppliers receive, so that the price suppliers will be asking for with tax $P_{sT}$ is the sum of the price given by the inverse supply curve $P_{s}$ in the tax: 

$$P_{sT} = P_{s} + T$$

Our system of equation now looks like this:

$$\left\{\begin{aligned}
P_d & = -a_d Q_d + b_d\\
P_{sT} & = a_s Q_s + b_s + T\\
\end{aligned}\right.$$

The new supply curve now has a higher intercept than before ($b_s + T$). If you look at the diagram, you will see that the supply curve is shifted upwards compared to the previous case.

If we want to solve the system of equations above to find the value of the unknown variables, the problem we are faced with is that we now have three unknowns ($P^*$, $Q^*$ and $T$) but still only two equations. Without more information, we won't be able to solve for all three variables simultaneously. That doesn't mean we're powerless, however. Assuming that markets clear, we can write:

$$\left\{\begin{aligned}
P^* & = -a_d Q^* + b_d\\
P^* & = a_s Q^* + b_s + T\\
\end{aligned}\right.$$

We then follow the exact same approach as before to solve this:

$$\begin{aligned}
a_s Q^* + b_s + T & = -a_d Q^* + b_d\\
(a_s+a_d)Q^* & = b_d - b_s - T\\
Q^* & = \frac{b_d - b_s}{a_s+a_d} - \frac{T}{a_s+a_d}\\
\end{aligned}$$

We can again plug in the numbers from the diagram to find our solution. Note that this time round, we don't actually get a number for $Q^*$, we get a linear equation that depends on $T$, which at this stage is still an unknown. This is because we don't have enough equations to do simultaneously find the values of $P^*$, $Q^*$ and $T$.

$$Q^* = \frac{25 -(-3)}{2+2}  - \frac{T}{2+2} = \frac{22}{4} - \frac{T}{4}- = 7 - \frac{T}{4}$$

What we do find, however, is at the equation for the equilibrium quantity depends negatively on the size of the tax $T$. Once we specify a level for the tax (say $T=4$), we can solve the equation above to get $Q*=6$, then replace in this the demand equation to get $P^*$:

$$P^* = -2\times 6 + 25 = -12 + 25 = 13$$

If we think of this in terms of systems of equations, what we are doing when we specifying the value of the tax, for example $T=4$, is essentially adding a 3rd equation to our system, which then has 3 equations for 3 unknowns and can be solved exactly. Even though $T=4$ is pretty trivial as far as equations go, it still counts: there is a left hand side, a right hand side and we are saying both sides are equal.

In [4]:
tax_widget(Tval_init = 4,Rflag_init = True)

### Understanding the impact of a tax

Note that an interesting prediction of the model is that the burden of the tax ends up shared between suppliers and consumers (toggle the 'show revenue button' to show/hide this). This is because despite the tax being charged to suppliers, they are able to pass on some of the tax to consumers. By the way, this would be the same if we had put the tax on consumers so that the price paid by consumers were increased $P_{dT} = P_{d} + T$. In this case, it is the the demand curve that would shift down, but by reducing their demand, the consumers would force suppliers to reduce their price and share the burden of the tax.

$$P_d  = -a_d Q_d + b_d - T$$

In the case of the parameters in the diagram, the revenue generated by the tax is shared evenly between suppliers and consumers. This does not have to be the case in general, however. A neat feature of our little system of equations is that it can help us predict the impact of a tax depending on the elasticities of supply and demand.

#### Case 1: Elastic demand, inelastic supply

Suppose that the demand curve for a given good is relatively price elastic, in other words, if the price goes up by 10%, the quantity demanded falls by more then 10%. On the other hand, the supply curve is price inelastic, so if price goes up by 10%, the quantity supplied goes up less than 10%. Examples of this type of markets are high-end agricultural produce like organic food, free-range eggs/meat, artisan/hand-crafted cheese, etc. On the demand side, they are relatively price-elastic. They tend to already be relatively expensive to begin with compared with cheaper alternatives, and increases in price lead to more-than-proportional decreases in quantity demanded. On the supply side, all agricultural produce tends to be price inelastic. The quantity supplied is governed by physical constraints like the time it takes to grow produce, the weather, the size of your field/orchard/herd of cattle that make it very difficult to change the size of your production in response to price changes.

So what happens if you put a tax on suppliers in in this kind of market? Well, as you can see below, the supplier is unable to pass the tax on to consumers and ends up impacted quite severely. So you really are taxing suppliers! The quantity does not change much, but the risk is that in the longer run farmers decide this is not a profitable activity so they simply give up and leave the market.

This situation is actually what UK agricultural producers are facing with Brexit arriving soon. The UK actually has a lot of high quality, high-welfare agricultural production, and a lot of it gets exported to the EU single market. If no trade deal is agreed and tariffs get imposed on UK exports, UK farmers will face a situation like the one shown below.

In [5]:
tax_widget(Qmax_init = 10, Pmax_init = 30, Tval_init = 10, a_d_init = 1.5,
    b_d_init = 25, a_s_init = 35, b_s_init = -175, Rflag_init = True, Lflag_init = True)

#### Case 2: Inelastic demand, elastic supply

This kind of case corresponds to a situation where the consumers will be prepared to buy a good regardless of its price (for example if it is a necessity that consumers cannot do without), but it is relatively easy for producers to increase or reduce their supply in response to price changes. A decent example would be electricity: it is hard to imagine living without it, so consumers will use it even if it is expensive, but as long as the overall demand does not exceed the generation capacity of suppliers, it is relatively easy for suppliers to spin up their generation if price increases. Furthermore, it is easy to imagine why we might want to tax suppliers: if they generate their electricity using fossil fuels (especially coal), then there is a very good environmental reason to tax suppliers, in order to try to discourage the use of fossil fuels. 

Unfortunately, as you can see from the diagram, all that happens is that nearly all the tax is passed on to consumers, and the quantity produced does not change much... So a tax alone is counterproductive. Instead, policy-makers may want to add a subsidy for clean generation (renewable sources), so that the market can offer a cheap alternative.


In [6]:
tax_widget(Qmax_init = 10, Pmax_init = 30, Tval_init = 10, a_d_init = 50,
    b_d_init = 275, a_s_init = 2, b_s_init = 5, Rflag_init = True, Lflag_init = True)

### The usefulness of simple examples

While the examples shown here are very simple, they provide two important illustrations:
1. First, they illustrate how economists use systems of equations. It is often the case that we start off with *more* unknowns than we have variables. What we can do, however, is decide that some of these are *policy* variables (taxes, spending, interest rates, etc...) and we can solve the system so that the unknowns that are **truly** unknown (price, quantity, GDP, inflation, etc...) are functions of these policy variables. That way, when we choose the level of the policy variables, we can figure out their impact on the other variables.
2. Second, they illustrate how even very simple models can nevertheless be very useful in shedding light on real-life situations. The model used here (supply/demand with a tax) is exactly the same in all the diagrams. As you can hopefully see, however, the story that you can tell changes dramatically depending on the values of the parameters. So play around with the parameters, and see if you can generate other interesting cases!
